In [7]:
!pip install beautifulsoup4

    100% |████████████████████████████████| 102kB 1.9MB/s a 0:00:01


In [8]:
from bs4 import BeautifulSoup

In [9]:
toronto_url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"

In [10]:
from requests import request

In [12]:
toronto_html = request('GET', toronto_url)

In [17]:
toronto_bs = BeautifulSoup(toronto_html.content)

In [24]:
toronto_bs.findAll(attrs={"class": "geo"})

[]

In [26]:
for tr in toronto_bs.tbody.findAll("tr"):
    

[<tr>
 <td style="text-align: left; vertical-align: top;">
 <h4><span id="Downtown_Core_.28Central.29"></span><span class="mw-headline" id="Downtown_Core_(Central)"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Core (Central)</a></span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_neighbourhoods_in_Toronto&amp;action=edit&amp;section=3" title="Edit section: Downtown Core (Central)">edit</a><span class="mw-editsection-bracket">]</span></span></h4>
 <ul><li><a href="/wiki/Alexandra_Park,_Toronto" title="Alexandra Park, Toronto">Alexandra Park</a></li>
 <li><a href="/wiki/The_Annex" title="The Annex">The Annex</a></li>
 <li><a href="/wiki/Baldwin_Village" title="Baldwin Village">Baldwin Village</a></li>
 <li><a href="/wiki/Cabbagetown,_Toronto" title="Cabbagetown, Toronto">Cabbagetown</a></li>
 <li><a href="/wiki/CityPlace,_Toronto" title="CityPlace, Toronto">CityPlace</a></li>
 <li><a href="/wiki/Chinato

In [38]:
table = toronto_bs.findAll("table")[6].tbody

In [42]:
len(table.findAll("tr")[1:])

33

In [43]:
import pandas as pd

In [68]:
base_url = "https://en.wikipedia.org"

,District,Neighbourhood,Latitude,Longitude


In [88]:
import numpy as np

In [95]:
from collections import defaultdict

In [97]:
nb_dict = defaultdict(list)

In [98]:
for row in table.findAll("tr")[1:]:
    district = row.td.text.strip()
    for nb in row.findAll("a"):
        nb_name = nb.text
        print(nb_name)
        nb_url = nb["href"]
        nb_req = request("GET", base_url + nb_url)
        nb_bs = BeautifulSoup(nb_req.content)
        try:
            coords = nb_bs.findAll("span", {"class": "geo"})[0].text
            coords = coords.split(";")
            lat = coords[0].strip()
            lon = coords[1].strip()
        except:
            lat = np.nan
            lon = np.nan
            
        
        nb_dict["District"].append(district)
        nb_dict["Neighbourhood"].append(nb_name)
        nb_dict["Latitude"].append(lat)
        nb_dict["Longitude"].append(lon)
        
        del nb_bs
        
    

In [105]:
df = pd.DataFrame(dict(nb_dict))

In [106]:
df

,District,Neighbourhood,Latitude,Longitude
0,C01,Downtown,43.6525028,-79.3835583
1,C01,Harbourfront,43.638,-79.385
2,C01,Little Italy,43.655,-79.413
3,C01,Little Portugal,43.65000,-79.43556
4,C02,The Annex,43.670,-79.404
5,C02,Yorkville,43.67028,-79.39111
6,C02,South Hill,43.681,-79.404
7,C02,Summerhill,43.683,-79.390
8,C02,Wychwood Park,43.67806,-79.41933
9,C02,Deer Park,43.68806,-79.394028


In [107]:
df.to_csv("toronto_districts_neighbourhoods_coordinates.cvs")

In [108]:
import folium

In [109]:
from geopy.geocoders import Nominatim

In [116]:
df[df.isna()]

,District,Neighbourhood,Latitude,Longitude
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [117]:
df

,District,Neighbourhood,Latitude,Longitude
0,C01,Downtown,43.6525028,-79.3835583
1,C01,Harbourfront,43.638,-79.385
2,C01,Little Italy,43.655,-79.413
3,C01,Little Portugal,43.65000,-79.43556
4,C02,The Annex,43.670,-79.404
5,C02,Yorkville,43.67028,-79.39111
6,C02,South Hill,43.681,-79.404
7,C02,Summerhill,43.683,-79.390
8,C02,Wychwood Park,43.67806,-79.41933
9,C02,Deer Park,43.68806,-79.394028


In [121]:
df[df["Latitude"].isna()]

,District,Neighbourhood,Latitude,Longitude
26,C08,St. Lawrence Market,NaN,NaN


In [122]:
address = "St. Lawrence Market, Toronto"

In [123]:
from geopy.geocoders import Nominatim

In [125]:
geoloc = Nominatim(user_agent="toronto_app")

In [127]:
loc = geoloc.geocode(address)

In [133]:
lm_lon = loc.longitude
lm_lat = loc.latitude

In [134]:
df.loc[26, "Longitude"] = lm_lon
df.loc[26, "Latitude"] = lm_lat

In [136]:
df.loc[26]

District                         C08
Neighbourhood    St. Lawrence Market
Latitude                     43.6492
Longitude                   -79.3717
Name: 26, dtype: object

In [137]:
import folium

In [138]:
toronto_loc = geoloc.geocode("Toronto, Canada")
toronto_lon = toronto_loc.longitude
toronto_lat = toronto_loc.latitude

In [169]:
df["Longitude"] = df["Longitude"].astype(float)
df["Latitude"] = df["Latitude"].astype(float)

In [197]:
toronto_map = folium.Map(location=(toronto_lat, toronto_lon), 
                         tiles="Stamen Toner",
                         zoom_start=10)

In [198]:
def add_circle(row):
    name = row["Neighbourhood"]
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    name_tooltip = folium.Tooltip(str(name))
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.7,
        tooltip=name_tooltip,
    ).add_to(toronto_map)

In [199]:
_ = df.apply(add_circle, axis=1)

In [200]:
toronto_map

In [184]:
df.to_csv("toronto.csv")